In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pydicom as dicom
import dicom2nifti
import nibabel as nib
import os

import pydicom

import glob
from tqdm import tqdm

import multiprocessing
import re

from nibabel.nicom.dicomwrappers import wrapper_from_file

tqdm.pandas()
np.set_printoptions(precision=3, suppress=True)


C:\Users\krish\AppData\Local\Temp\ipykernel_55456\407344333.py:18: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  from nibabel.nicom.dicomwrappers import wrapper_from_file


In [2]:
img_directories = '../../Simple Path Dataset/manifest-1654812109500/'
annotation_boxes = pd.read_excel("../../Simple Path Dataset/Annotation_Boxes.xlsx").set_index("Patient ID")

FILE_EXT = '.img.gz'            # fileextension for nifti

# for dicom2nifti
transfer_directory = '../../Processed NIFTI Dataset/'

seg_annotations = pd.read_csv('segmentation_annotations.csv')

In [3]:
# file_descriptions = pd.read_excel("../../Simple Path Dataset/Breast-Cancer-MRI-filepath_filename-mapping.xlsx")
# file_descriptions = file_descriptions[['sop_instance_UID', 'original_path_and_filename', 'classic_path', 'descriptive_path']]

# # process strings to get required info: patient ID, sequence, description, and dicom paths

# file_descriptions['patient'] = file_descriptions['classic_path'].apply(lambda x: x.split('/')[1])
# file_descriptions['sequence'] = file_descriptions['classic_path'].apply(lambda x: x.split('/')[3])
# file_descriptions['description'] = file_descriptions['original_path_and_filename'].apply(lambda x: x.split('/')[2])
# file_descriptions['dicom_paths'] = file_descriptions['classic_path'].apply(lambda x: '/'.join(x.split('/')[:-1]))

In [4]:
# file_descriptions.description.unique()

In [5]:
# #extract relevant file descriptions, pre and post_1 from simple file paths
# relevant_file_descriptions = file_descriptions[file_descriptions['description'].isin(['post_1', 'pre'])]
# relevant_data = relevant_file_descriptions[['patient', 'sequence', 'dicom_paths', 'description']].drop_duplicates().reset_index(drop = True)
# relevant_data['dicom_paths'] = relevant_data['dicom_paths'].apply(lambda x: os.path.join(img_directories, x))
# relevant_data['patient_dir'] = relevant_data['patient'].apply(lambda x: os.path.join(transfer_directory, x))
# relevant_data['output_paths'] = relevant_data.apply(lambda x: os.path.join(x['patient_dir'], x['description']) + FILE_EXT, axis = 1)
# relevant_data.to_csv("relevant_data.csv", index = False)

In [6]:
relevant_data = pd.read_csv("relevant_data.csv")
relevant_data

,patient,sequence,dicom_paths,description,patient_dir,output_paths
0,Breast_MRI_001,1.3.6.1.4.1.14519.5.2.1.1514720349923003958578...,../../Simple Path Dataset/manifest-16548121095...,post_1,../../Github NIFTI/Breast_MRI_001,../../Github NIFTI/Breast_MRI_001\post_1.nii.gz
1,Breast_MRI_001,1.3.6.1.4.1.14519.5.2.1.1857778498036652445367...,../../Simple Path Dataset/manifest-16548121095...,pre,../../Github NIFTI/Breast_MRI_001,../../Github NIFTI/Breast_MRI_001\pre.nii.gz
2,Breast_MRI_002,1.3.6.1.4.1.14519.5.2.1.1702386334115195086234...,../../Simple Path Dataset/manifest-16548121095...,post_1,../../Github NIFTI/Breast_MRI_002,../../Github NIFTI/Breast_MRI_002\post_1.nii.gz
3,Breast_MRI_002,1.3.6.1.4.1.14519.5.2.1.2920879504444133146439...,../../Simple Path Dataset/manifest-16548121095...,pre,../../Github NIFTI/Breast_MRI_002,../../Github NIFTI/Breast_MRI_002\pre.nii.gz
4,Breast_MRI_003,1.3.6.1.4.1.14519.5.2.1.3339321204011089072051...,../../Simple Path Dataset/manifest-16548121095...,post_1,../../Github NIFTI/Breast_MRI_003,../../Github NIFTI/Breast_MRI_003\post_1.nii.gz
...,...,...,...,...,...,...
1839,Breast_MRI_920,1.3.6.1.4.1.14519.5.2.1.2243370307638913554367...,../../Simple Path Dataset/manifest-16548121095...,pre,../../Github NIFTI/Breast_MRI_920,../../Github NIFTI/Breast_MRI_920\pre.nii.gz
1840,Breast_MRI_921,1.3.6.1.4.1.14519.5.2.1.2153335720893817815538...,../../Simple Path Dataset/manifest-16548121095...,post_1,../../Github NIFTI/Breast_MRI_921,../../Github NIFTI/Breast_MRI_921\post_1.nii.gz
1841,Breast_MRI_921,1.3.6.1.4.1.14519.5.2.1.2995579324872770731987...,../../Simple Path Dataset/manifest-16548121095...,pre,../../Github NIFTI/Breast_MRI_921,../../Github NIFTI/Breast_MRI_921\pre.nii.gz
1842,Breast_MRI_922,1.3.6.1.4.1.14519.5.2.1.5039771361051598068760...,../../Simple Path Dataset/manifest-16548121095...,post_1,../../Github NIFTI/Breast_MRI_922,../../Github NIFTI/Breast_MRI_922\post_1.nii.gz


In [7]:
def dicom_to_volume(dicom_series):
    """Convert a DICOM series into a float32 volume with orientation.
    The input should be a list of 'dataset' objects from pydicom.
    The output is a tuple (voxel_array, voxel_spacing, affine_matrix)
    """
    # Create numpy arrays for volume, pixel spacing (ps),
    # slice position (ipp or ImagePositionPatient), and
    # slice orientation (iop or ImageOrientationPatient)
    n = len(dicom_series)
    shape = (n,) + dicom_series[0].pixel_array.shape
    vol = np.empty(shape, dtype=np.float32)
    ps = np.empty((n,2), dtype=np.float64)
    ipp = np.empty((n,3), dtype=np.float64)
    iop = np.empty((n,6), dtype=np.float64)

    for i, ds in enumerate(dicom_series):
        # create a single complex-valued image from real,imag
        image = ds.pixel_array
        try:
            slope = float(ds.RescaleSlope)
        except (AttributeError, ValueError):
            slope = 1.0
        try:
            intercept = float(ds.RescaleIntercept)
        except (AttributeError, ValueError):
            intercept = 0.0
        vol[i,:,:] = image*slope + intercept
        ps[i,:] = dicom_series[i].PixelSpacing
        ipp[i,:] = dicom_series[i].ImagePositionPatient
        iop[i,:] = dicom_series[i].ImageOrientationPatient

    # create nibabel-style affine matrix and pixdim
    # (these give DICOM LPS coords, not NIFTI RAS coords)
    affine, pixdim = create_affine(ipp, iop, ps)
    return vol, pixdim, affine

def create_affine(ipp, iop, ps):
    """Generate an affine matrix from DICOM IPP and IOP attributes.
    The ipp (ImagePositionPatient) parameter should an Nx3 array, and
    the iop (ImageOrientationPatient) parameter should be Nx6, where
    N is the number of DICOM slices in the series.
    The return values are the affine matrix and the pixdim.
    Note the the output will use DICOM anatomical coordinates:
    x increases towards the left, y increases towards the back.
    """
    # solve Ax = b where x is slope, intecept
    n = ipp.shape[0]
    A = np.column_stack([np.arange(n), np.ones(n)])
    x, r, rank, s = np.linalg.lstsq(A, ipp, rcond=None)
    # round small values to zero
    x[(np.abs(x) < 1e-6)] = 0.0
    vec = x[0,:] # slope
    pos = x[1,:] # intercept

    # pixel spacing should be the same for all image
    spacing = np.ones(3)
    spacing[1::-1] = ps[0,:]
    if np.sum(np.abs(ps - spacing[1::-1])) > spacing[0]*1e-6:
        print("Pixel spacing is inconsistent!\n");

    # compute slice spacing, round to 7 decimal places
    spacing[2] = np.round(np.sqrt(np.sum(np.square(vec))), 7)

    # get the orientation
    iop_average = np.mean(iop, axis=0)
    u = iop_average[0:3]
    u /= np.sqrt(np.sum(np.square(u)))
    v = iop_average[3:6]
    v /= np.sqrt(np.sum(np.square(v)))

    # round small values to zero
    u[(np.abs(u) < 1e-6)] = 0.0
    v[(np.abs(v) < 1e-6)] = 0.0

    # create the matrix
    mat = np.eye(4)
    mat[0:3,0] = u*spacing[0]
    mat[0:3,1] = v*spacing[1]
    mat[0:3,2] = vec
    mat[0:3,3] = pos

    # check whether slice vec is orthogonal to iop vectors
    dv = np.dot(vec, np.cross(u, v))
    qfac = np.sign(dv)
    if np.abs(qfac*dv - spacing[2]) > 1e-6:
        print("Non-orthogonal volume!\n");

    # compute the pixdim array, with qfac as the first element
    pixdim = np.hstack([np.array(qfac), spacing])

    return mat, pixdim

In [128]:
# take one patient
row = relevant_data[relevant_data['description'] == 'post_1'].iloc[5]
patient = row['patient']
dicom_path = row['dicom_paths']
filetype = row['description']

nifti_path = os.path.join(transfer_directory, patient, filetype) + FILE_EXT
nifti_path

'../../Processed NIFTI Dataset/Breast_MRI_006\\post_1.img.gz'

In [129]:
dcm_files = glob.glob(os.path.join(dicom_path, '*.dcm'))
dcm_file = dcm_files[0]

dcms = sorted([pydicom.dcmread(x) for x in dcm_files], key =lambda x: int(x.InstanceNumber))

dcm = wrapper_from_file(dcm_file)
dcm_pydicom = pydicom.dcmread(dcm_file)
nifti = nib.nifti1.load(nifti_path)

# dcm = dicom.dcmread(dcm_file)

_, _, aff_dcm = dicom_to_volume(dcms)

In [130]:
dcm.affine

array([[   0.   ,   -0.684,   -0.   ,  178.305],
       [  -0.684,    0.   ,   -0.   ,  220.512],
       [   0.   ,    0.   ,   -1.   , -100.817],
       [   0.   ,    0.   ,    0.   ,    1.   ]])

In [131]:
nifti.affine

array([[   0.684,    0.   ,   -0.   , -178.305],
       [   0.   ,    0.684,   -0.   , -220.512],
       [   0.   ,    0.   ,    1.   , -100.817],
       [   0.   ,    0.   ,    0.   ,    1.   ]])

In [132]:
# # Extract IPP, IOP, and Pixel Spacing
# ipp = np.array(dcm_pydicom.ImagePositionPatient)
# iop = np.array(dcm_pydicom.ImageOrientationPatient).reshape(2, 3)
# pixel_spacing = np.array(dcm_pydicom.PixelSpacing)

# # Construct the DICOM to patient coordinate system transformation matrix
# dicom_to_patient_matrix = np.zeros((4, 4))


# dicom_to_patient_matrix[:3, :3] = np.hstack((iop[:, 0:1] * pixel_spacing[0], iop[:, 1:2] * pixel_spacing[1], np.cross(iop[:, 0], iop[:, 1]).reshape(3, 1)))
# dicom_to_patient_matrix[:3, 3] = ipp
# dicom_to_patient_matrix[3, 3] = 1

In [133]:
aff_nif = nifti.affine
# aff_dcm = dcm.affine

In [134]:
aff_dcm

array([[  -0.684,    0.   ,    0.   ,  178.305],
       [   0.   ,   -0.684,    0.   ,  220.512],
       [   0.   ,    0.   ,    1.   , -100.817],
       [   0.   ,    0.   ,    0.   ,    1.   ]])

In [135]:
vox_transform = np.rint(np.linalg.inv(aff_nif).dot(aff_dcm))
vox_transform

array([[ -1.,   0.,   0., 522.],
       [  0.,  -1.,   0., 645.],
       [  0.,   0.,   1.,   0.],
       [  0.,   0.,   0.,   1.]])

In [136]:
bb_details = annotation_boxes.loc[patient]
start_row, end_row = bb_details['Start Row'], bb_details['End Row']
start_col, end_col = bb_details['Start Column'], bb_details['End Column']
start_slc, end_slc = bb_details['Start Slice'], bb_details['End Slice']

corner1 = np.array([start_col, start_row, start_slc])
corner2 = np.array([end_col, end_row, end_slc])

trans_corner1 = np.rint(nib.affines.apply_affine(vox_transform, corner1))
trans_corner2 = np.rint(nib.affines.apply_affine(vox_transform, corner2))

# trans_corner1 = vox_transform@corner1
# trans_corner2 = vox_transform@corner2

In [137]:
start_slc, end_slc

(39, 48)

In [138]:
corner1

array([408, 331,  39], dtype=int64)

In [139]:
corner2

array([434, 371,  48], dtype=int64)

In [140]:
trans_corner1

array([114., 314.,  39.])

In [141]:
trans_corner2

array([ 88., 274.,  48.])

In [142]:
upper_corner = np.ceil(np.maximum(trans_corner1, trans_corner2))
lower_corner = np.floor(np.minimum(trans_corner1, trans_corner2))

In [143]:

srow, scol, sslice = lower_corner
erow, ecol,  eslice = upper_corner

In [144]:
if sslice <= 0:
    sslice, eslice = nifti.dataobj.shape[-1] + sslice, nifti.dataobj.shape[-1] + eslice

In [145]:
# flip along y-axis, then rotate counter-clockwise
scol, ecol = nifti.dataobj.shape[0] - end_col, nifti.dataobj.shape[0] - start_col

# x, y -> y, -x
scol, ecol, srow, erow = start_row, end_row, nifti.dataobj.shape[1] - scol, nifti.dataobj.shape[1] - ecol, 

sslice, eslice = start_slc, end_slc

In [146]:
# lower_corner

In [147]:
sslice, eslice, patient

(39, 48, 'Breast_MRI_006')

In [148]:
from matplotlib import patches

def plot(idx):
    fig, ax = plt.subplots(1, 1, figsize = (6, 6))
    ax.imshow(nifti.dataobj[:, :, idx], cmap = 'gray')
    ax.add_patch(patches.Rectangle((scol, srow), ecol-scol, erow - srow, fill = False, edgecolor = 'red', lw = 1))

In [149]:
from ipywidgets import Dropdown, IntSlider, Label
from traitlets import directional_link
from IPython.display import display
import ipywidgets as widgets

slice_max = nifti.dataobj.shape[-1]-1
b = IntSlider(min = 0, max = slice_max, value = (sslice + eslice)//2)
widgets.interact(plot, idx = b)

interactive(children=(IntSlider(value=43, description='idx', max=147), Output()), _dom_classes=('widget-intera…

<function __main__.plot(idx)>